In [1]:
import json
import tweepy
from tweepy import OAuthHandler, Stream
from tweepy.streaming import StreamListener
from kafka import KafkaProducer, KafkaConsumer
from datetime import datetime
import time

In [2]:
topic = 'testing'
word_filter = ['spotify com track']
# language_filter = ['en']

access_token = '' 
access_secret = '' 
consumer_key = ''         
consumer_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [5]:
class KafkaPushListener(StreamListener):
    def __init__(self, time_limit = 3600):
        print("Publish data to topic: "+topic)
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092')
        self.start_time = time.time()
        self.limit = time_limit
        
    def normalize_timestamp(self, time):
        mytime = datetime.strptime(time, "%a %b %d %H:%M:%S +0000 %Y")
        return(mytime.strftime("%Y-%m-%d %H:%M:%S"))
    
    def on_data(self, data):
        json_object = json.loads(data)
        if (time.time() - self.start_time) < self.limit:
            if len(json_object['entities']['urls']) > 0:
                record = ''
                record += str(self.normalize_timestamp(str(json_object['created_at'])))
                record += '|'
                record += str(json_object['user']['screen_name'].encode('utf-8'))
                record += '|'
                record += str(json_object['entities']['urls'][0]['expanded_url'])
                self.producer.send(topic, str.encode(record))
                print(record)
            return True
        else:
            self.producer.close()
            return False

In [6]:
if __name__ == '__main__':
    twitter_stream = Stream(auth, KafkaPushListener())
    twitter_stream.filter(track=word_filter)

Publish data to topic: testing
2020-09-25 03:55:29|b'phantasmofist'|https://open.spotify.com/track/1Cwsd5xI8CajJz795oy4XF?si=HuYQjjv3Rz26vKZoKEQOlQ
2020-09-25 03:55:29|b'chenvisual'|https://twitter.com/i/web/status/1309340722617868289
2020-09-25 03:55:29|b'filppa_petri'|https://open.spotify.com/track/1gmXQGru8mdJ1x19RzoUvA?si=1m9FUdBBRYqLGAJ9QplMHg
2020-09-25 03:55:29|b'RebeccaCristi20'|https://twitter.com/i/web/status/1309340723070803968
2020-09-25 03:55:29|b'DylanCrozier_'|https://open.spotify.com/track/3XV5r1cMB2AlD2ir0v6mfm?si=aRZCx8bfROa0WWkKODwVJQ
2020-09-25 03:55:29|b'raimlchr'|https://open.spotify.com/track/1dLXLETYiZohIZWnse4ypG?si=pPPY7TybTFSnPEcyk8j3Yg
2020-09-25 03:55:29|b'directionbabys2'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:30|b'chenleyes'|https://open.spotify.com/track/2sOz5KStj3M1L1ErNo6MCC?si=yfld2bDzQh-g6NYuITF1dg
2020-09-25 03:55:30|b'ClipsCray'|https://open.spotify.com/track/7sa8AlYpSZDDFQpQCDXq82?si=_evbm7

2020-09-25 03:55:37|b'onIyhess'|http://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=nJ6rkAu3QyuBeDLh36xgDQ
2020-09-25 03:55:37|b'Hayatowhat'|https://open.spotify.com/track/4RHyIzFRht2z17XeHB9dsF?si=j2uF6dq8TGW4hzLjP02jTQ
2020-09-25 03:55:38|b'cheryiixing'|https://open.spotify.com/track/53YZi9zgTnF0Py0K6ejyWz?si=wwcmURzSTjW_SBWvOK8WWw
2020-09-25 03:55:38|b'p23dro'|https://open.spotify.com/track/1cYmwcMFO7SjXiJmCgzsHj?si=8Dg108egQR6AtOEKvMx6LQ
2020-09-25 03:55:38|b'demonsruel'|https://open.spotify.com/album/5EzDhyNZuO7kuaABHwbBKX?si=a_DzMCm_Su2fsP2gBW8tEg
2020-09-25 03:55:38|b'spacealvex'|https://open.spotify.com/track/4A8pBxx0of2Y8r56dT0atf?si=Rag2fRQSS3G90pyIA3TNgA
2020-09-25 03:55:38|b'peraltarings'|https://open.spotify.com/track/5WokFKscrfGEGGLPTu3jgO?si=JdLbSWnfQQuswFP1NhVvcQ
2020-09-25 03:55:38|b'ultravioletmuse'|https://open.spotify.com/album/75sm9IEmN01J5MsyZr4Cw1?si=BJWgLe5RTUaTQS2PGuejNw
2020-09-25 03:55:38|b'sinsntragediess'|https://open.spotify.com/track/1IP2NS7ov1MAh2Y4N

2020-09-25 03:55:45|b'jenniecolour'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:45|b'vineysgf'|https://open.spotify.com/track/2Qt8YTTXdgQva0Vv9VjBlS?si=NCCGssuETHu-y5wWBpH20Q
2020-09-25 03:55:46|b'cinnamoonm'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:46|b'4misfitts'|https://open.spotify.com/track/1MONUudxAjEk76FJvzGhuD?si=7szL_pxQQQKCQtab_RHt7A
2020-09-25 03:55:46|b'lucazabreu'|https://twitter.com/i/web/status/1309340793811931136
2020-09-25 03:55:46|b'claraaa68650439'|https://open.spotify.com/track/6cWKFEOVJvtQQ3B1tiT0tr?si=hlzKmluDTgaJ6F-vubYlSQ
2020-09-25 03:55:46|b'listendestro'|https://open.spotify.com/track/1ASx2Qk52e5rlXgtWIKYCA?si=M0SlEJkATOeDKg7x6GXIww
2020-09-25 03:55:46|b'icarxsmalik'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:46|b'byuncutton'|https://open.spotify.com/track/53YZi9zgTnF0Py0K6ejyWz?si=x5rw1qTvT0SQdkBL9

2020-09-25 03:55:53|b'hesfoIklore'|https://open.spotify.com/track/6pUEp8lDzjN2Jv4BdM9B74?si=5ywnWsj4Q_C7zYSLMu5uow
2020-09-25 03:55:53|b'drunk93s'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:53|b'BorsenGelaber'|https://open.spotify.com/episode/32utGlJD1pyGGVXzsiGuxO?si=5rzkOILfSMewTKm8W-6DSA
2020-09-25 03:55:53|b'uwustin'|https://open.spotify.com/episode/2tOq8uTF0lERaKCd339G8Y?si=Z290LNw3SvuQKgC58K5t1A
2020-09-25 03:55:53|b'twdxbizzle'|https://open.spotify.com/track/40iocOzytrqaRn5RbGPDNp?si=A5x4eIy_SHiWW0J1NdLxaA
2020-09-25 03:55:54|b'mygutjang'|https://open.spotify.com/track/4fAI0wdVepOCi4xRwj40z9?si=O7PM6K3FTZG3V6sISzfFkA
2020-09-25 03:55:53|b'HanielSilva177'|https://twitter.com/i/web/status/1309340826112229378
2020-09-25 03:55:54|b'baineslynch'|https://twitter.com/i/web/status/1309340827483860993
2020-09-25 03:55:54|b'favzpankow'|https://twitter.com/i/web/status/1309340828612124673
2020-09-25 03:55:54|b'wabteass'|https://open.spo

2020-09-25 03:56:03|b'ycngke'|https://twitter.com/i/web/status/1309340864078987264
2020-09-25 03:56:03|b'nlpieroni'|https://open.spotify.com/artist/1on7ZQ2pvgeQF4vmIA09x5?si=6kWFPURLRgayYfEJ82MgXw
2020-09-25 03:56:03|b'mindwmi'|https://open.spotify.com/track/1dLXLETYiZohIZWnse4ypG?si=V1N10XC1S5-hcWUaD2gazg
2020-09-25 03:56:03|b'exomysirius'|https://open.spotify.com/track/53YZi9zgTnF0Py0K6ejyWz?si=x5rw1qTvT0SQdkBL9Mcgcg
2020-09-25 03:56:03|b'favzpankow'|https://twitter.com/i/web/status/1309340865098403842
2020-09-25 03:56:03|b'sweetlouwtt'|https://open.spotify.com/track/1dLXLETYiZohIZWnse4ypG?si=0o-tR8xbQaKxABj2WxBpsQ
2020-09-25 03:56:03|b'VeraOlortegui'|https://open.spotify.com/track/5u1n1kITHCxxp8twBcZxWy?si=vSLdi0lIRFmkZUCYkddgqQ
2020-09-25 03:56:03|b'frueity'|https://twitter.com/i/web/status/1309340867421896704
2020-09-25 03:56:03|b'warmcherub'|https://open.spotify.com/track/6lMMiFxOnNZBEJFc4UQPwS?si=mtLc5BfhTziGc3mF8hFpLw
2020-09-25 03:56:03|b'HanaBab11906342'|https://open.spotify.

2020-09-25 03:56:12|b'asuka_engage'|https://twitter.com/i/web/status/1309340902385569792
2020-09-25 03:56:12|b'walldanceski'|https://open.spotify.com/track/3SXqbJNTe2s1xePDzYwkZm?si=Fz02b9n7RuenH3DsMdXgpg
2020-09-25 03:56:12|b'dyingfirst1'|https://open.spotify.com/track/7D0VClhoNONSsdPYWwcTej?si=MVMxcI4kRMqioef9DApKjA
2020-09-25 03:56:12|b'fuckflearh'|https://open.spotify.com/track/6Q5mJdY0XcBLNVq9NLZuto?si=N8UDi_NkQS2Ic1p4hmnTcQ
2020-09-25 03:56:12|b'xia0hua'|https://open.spotify.com/track/6zeeWid2sgw4lap2jV61PZ?si=3Dme1-VHQTmF-HedUT1uFQ
2020-09-25 03:56:12|b'vanessa_issa'|https://open.spotify.com/track/4UpKhXsRaQlzcJ7CglLPlI?si=dfY_oP88T-moQtzX29hphw
2020-09-25 03:56:12|b'000L100'|https://open.spotify.com/track/0v1x6rN6JHRapa03JElljE?si=D2gSY3HcQbCdpvqPwmFx-Q
2020-09-25 03:56:13|b'gookierume'|https://open.spotify.com/track/0pYacDCZuRhcrwGUA5nTBe?si=2fUnOaI0S16terff-znOkw&context=spotify%3Aplaylist%3A37i9dQZF1E39kxcHhySrFN
2020-09-25 03:56:13|b'hoesokehoe'|https://open.spotify.com/tra

2020-09-25 03:56:20|b'IiI_violence'|https://open.spotify.com/socialsession/WE9vB5ZszErrDPvC1FHCpOyKewuTMoSlcatJSI3mxQ0cbUIufGYH3uUKsECIG5ulTOJGPZqg5z2UxN0PvTdpVGROc0lvymIGPR6WBQsxD8e?si=DvoRidd3Rh2867YrTd1eBA
2020-09-25 03:56:20|b'OttoJoseM'|https://open.spotify.com/track/5HNCy40Ni5BZJFw1TKzRsC?si=1QvCjp8GTD2cs0hwCXfmnw
2020-09-25 03:56:20|b'ichusmind'|https://open.spotify.com/track/01G4lEhmfh1NTzq7kKK38o?si=0rrKJrODR1Keat6ANKoCYQ
2020-09-25 03:56:20|b'painhanxqu1'|https://open.spotify.com/track/2hgk8iKN9UdoM4ILlJ9F5N?si=mUM1g54IQ8-TrL3WVB5XQg
2020-09-25 03:56:20|b'jeonsmel'|https://open.spotify.com/track/7D0VClhoNONSsdPYWwcTej?si=pZyFCGuDSvSXLaIq9s7jIA
2020-09-25 03:56:21|b'czlifes'|https://open.spotify.com/track/786H6CRL3H5WNPnkqTfnNX?si=gvgufaApRdi6DQsHdo0eHg
2020-09-25 03:56:21|b'aynahazirah'|https://open.spotify.com/track/2CSrG5fnBpcdfPXg7sJUBi?si=4_PZIvjgTGicBJEG1fVDcw
2020-09-25 03:56:21|b'wHenG_09'|https://open.spotify.com/playlist/5mnaJLV45eBS2n6u6iuYA9?si=kiag4teFSY6oi5o07b8C